In [ ]:
key="7NvUEUX4tnzOja5KQ99gmUG37DQOV9oelvz1akWAr2Zts9X57djRMwbvfgjQoykp"
secret="X9CWCXNsdypjEU8Q0AQaoaqPrcnaX4wpDe5KVxsAfThkVJJAvufiGJ3tb95QqnQC"

# key api test2
import time



In [ ]:
from binance_ft.um_futures import UMFutures
um_futures_client = UMFutures(key=key, secret=secret)
# um_futures_client = UMFutures(key=key, secret=secret, base_url="https://testnet.binancefuture.com")

from binance.spot import Spot
spot_client = Spot(api_key=key, api_secret=secret)

In [30]:

pair = "GMTUSDT"
pair_spot = pair.replace("USDT", "USDT")

future_info = um_futures_client.exchange_info()['symbols']
kk = [i['filters'] for i in future_info if i['symbol'] == pair][0][1]
print("future_info", kk)

print("spot_info", spot_client.exchange_info(pair_spot)['symbols'][0]['filters'][1])

bid_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])
ask_spot = float(spot_client.book_ticker(pair_spot)['askPrice'])

mark_price = float(spot_client.ticker_price(pair_spot)['price'])
mark_ft = float(um_futures_client.mark_price(pair)['markPrice'])
bid_spot, mark_price, ask_spot, mark_ft


future_info {'stepSize': '1', 'maxQty': '10000000', 'filterType': 'LOT_SIZE', 'minQty': '1'}
spot_info {'filterType': 'LOT_SIZE', 'minQty': '0.10000000', 'maxQty': '92141578.00000000', 'stepSize': '0.10000000'}


(0.4061, 0.4061, 0.4062, 0.40593676)

In [41]:
# spot truoc future sau:
precision_ft = 0
precision_spot =1
um_futures_client.change_leverage(symbol=pair, leverage=4, recvWindow=6000)


bid_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])
ask_spot = float(spot_client.book_ticker(pair_spot)['askPrice'])
mark_spot = float(spot_client.ticker_price(pair_spot)['price'])
price_spot = mark_spot
print("order spot at", price_spot)

usdt = 11
quantity_spot = round(usdt/price_spot,precision_spot)
# if quantity_ft < quantity_spot:
#     quantity_ft += 0.1

open_spot = spot_client.new_order(symbol=pair_spot, side="BUY", type="LIMIT", quantity=quantity_spot, price=price_spot, timeInForce="GTC")

order_id_spot = open_spot["orderId"]
while True:
    res_query_spot = spot_client.get_order(symbol=pair_spot, orderId=order_id_spot, recvWindow=6000)
    if res_query_spot['status'] == "FILLED":
        executedQty_spot = float(res_query_spot['executedQty'])
        usdt_value_spot = float(res_query_spot['cummulativeQuoteQty'])
        price_spot = float(res_query_spot['price'])
        # print("spot:", price_spot, usdt_value_spot, "quantity:", executedQty_spot, res_query_spot['type'])
        print("price_spot =", price_spot)
        print("usdt_value_spot =",usdt_value_spot)
        print("quantity_spot =", executedQty_spot)
        print("type_spot =",res_query_spot['type'])
        print("---------------------")
        now_ft_price = float(um_futures_client.mark_price(pair)['markPrice']) 
        now_ft_price = round(now_ft_price, 4)
        # round(usdt_value_spot/now_ft_price, 1)
        quantity_ft = round((usdt)/now_ft_price, precision_ft)
        open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="LIMIT", quantity=quantity_ft, price = now_ft_price, timeInForce="GTC")

        # open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="MARKET", quantity=quantity_ft)
        ft_order_id = open_future["orderId"]
        print("ft_order_id =", ft_order_id)
        while True:
            res_query_ft = um_futures_client.query_order(symbol=pair, orderId=ft_order_id, recvWindow=6000)
            if res_query_ft['status'] == "FILLED":
                # avg_spot = float(spot_order['price'])
                price_ft = float(res_query_ft['avgPrice'])
                executedQty_future = float(res_query_ft['executedQty'])
                usdt_value_ft = float(res_query_ft['cumQuote'])
                print("price_ft =", price_ft)
                print("usdt_value_ft =",usdt_value_ft)
                print("quantity_ft =", executedQty_future)
                print("type_ft =",res_query_ft['type'])
                break
            time.sleep(0.1)
        break
    time.sleep(0.1)

order spot at 0.4215
price_spot = 0.4215
usdt_value_spot = 11.00115
quantity_spot = 26.1
type_spot = LIMIT
---------------------
ft_order_id = 13259263572
price_ft = 0.4216
usdt_value_ft = 10.9616
quantity_ft = 26.0
type_ft = LIMIT


In [64]:
# future truoc spot sau:
precision_ft = 0
precision_spot =1
usdt = 12
um_futures_client.change_leverage(symbol=pair, leverage=5, recvWindow=6000)

# print("order spot at", price_spot)


now_ft_price = float(um_futures_client.mark_price(pair)['markPrice']) 
now_ft_price = round(now_ft_price, 4)
price_ft = now_ft_price+0.0004
print("order ft at", price_ft)

quantity_ft = round((usdt)/now_ft_price, precision_ft)
open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="LIMIT", quantity=quantity_ft, price = now_ft_price, timeInForce="GTC")

# open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="MARKET", quantity=quantity_ft)
ft_order_id = open_future["orderId"]
print("ft_order_id =", ft_order_id)
while True:
    res_query_ft = um_futures_client.query_order(symbol=pair, orderId=ft_order_id, recvWindow=6000)
    if res_query_ft['status'] == "FILLED":
        # avg_spot = float(spot_order['price'])
        price_ft = float(res_query_ft['avgPrice'])
        executedQty_future = float(res_query_ft['executedQty'])
        usdt_value_ft = float(res_query_ft['cumQuote'])
        print("price_ft =", price_ft)
        print("usdt_value_ft =",usdt_value_ft)
        print("quantity_ft =", executedQty_future)
        print("type_ft =",res_query_ft['type'])
        print("---------------------")
        price_spot = float(spot_client.ticker_price(pair_spot)['price'])
        quantity_spot = round(usdt_value_ft/price_spot,precision_spot)
        open_spot = spot_client.new_order(symbol=pair_spot, side="BUY", type="LIMIT", quantity=quantity_spot, price=price_spot, timeInForce="GTC")
        order_id_spot = open_spot["orderId"]
        while True:
            res_query_spot = spot_client.get_order(symbol=pair_spot, orderId=order_id_spot, recvWindow=6000)
            if res_query_spot['status'] == "FILLED":
                executedQty_spot = float(res_query_spot['executedQty'])
                usdt_value_spot = float(res_query_spot['cummulativeQuoteQty'])
                price_spot = float(res_query_spot['price'])
                # print("spot:", price_spot, usdt_value_spot, "quantity:", executedQty_spot, res_query_spot['type'])
                print("price_spot =", price_spot)
                print("usdt_value_spot =",usdt_value_spot)
                print("quantity_spot =", executedQty_spot)
                print("type_spot =",res_query_spot['type'])
                break
            time.sleep(0.1)
        break
    time.sleep(0.1)

order ft at 0.4385
ft_order_id = 13263165204
price_ft = 0.4381
usdt_value_ft = 11.8287
quantity_ft = 27.0
type_ft = LIMIT
---------------------
price_spot = 0.4382
usdt_value_spot = 11.8314
quantity_spot = 27.0
type_spot = LIMIT


In [ ]:
# spot_client.cancel_open_orders(symbol=pair_spot)

In [67]:

fee_open = 0.00075 # use busd
fee_close = 0.00075

fee_ft_open = 0.00017
fee_ft_close = 0.00017
all_fee_per = fee_open + fee_close +fee_ft_open+fee_ft_close



In [43]:
fee_spot = (fee_open + fee_close)*usdt_value_spot
fee_future = (fee_ft_open + fee_ft_close) * usdt_value_ft
print("fee spot", fee_spot)
print("fee_future", fee_future)
while True:
    # curret_ft = float(um_futures_client.mark_price(pair)['markPrice'])
    # curret_spot = float(spot_client.ticker_price(pair_spot)['price'])

    curret_ft = float(um_futures_client.book_ticker(pair)['askPrice'])  # > mark price
    curret_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])  # < mark price

    gap_ft = -(curret_ft - price_ft)/price_ft * 100
    gap_spot = (curret_spot - price_spot)/price_spot * 100  

    if gap_spot + gap_ft > 0.25:
        print(gap_spot + gap_ft)

    pnl_ft = (price_ft - curret_ft)/price_ft * usdt_value_ft
    pnl_spot = (curret_spot-price_spot)/price_spot * usdt_value_spot
    # print("pnl ft: " ,pnl_ft, "pnl_spot", pnl_spot)
    print("pnl:",pnl_spot + pnl_ft - fee_spot - fee_future)
    if pnl_spot + pnl_ft - fee_spot - fee_future > usdt_value_spot * 0.0005 :
        # close position
        close_future =  um_futures_client.new_order(symbol=pair, side="BUY", type="LIMIT", quantity=quantity_ft, price = curret_ft, timeInForce="GTC")
        close_spot = spot_client.new_order(symbol=pair_spot, side="SELL", type="LIMIT", quantity=quantity_spot, price=curret_spot, timeInForce="GTC")
        close_future_id=close_future['orderId']
        close_spot_id=close_spot['orderId']

        
        # close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
        # close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)
        print("*****************************")
        print("curret_ft", curret_ft)
        print("curret_spot", curret_spot)
        time.sleep(1)
        print("take order at:")
        while True:
            status_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future_id)
            status_spot_close = spot_client.get_order(symbol=pair_spot, orderId=close_spot_id, recvWindow=6000)
            if status_spot_close['status'] == "FILLED" and len(status_ft_close):
                price_ft_close_real = float(status_ft_close[0]['price'])
                # price_spot_close_real = float(close_spot['fills'][0]['price'])
                price_spot_close_real = curret_spot
                print("ft:", price_ft_close_real)
                print("spot:", price_spot_close_real)
                break
            else:
                time.sleep(1)

        break
    time.sleep(0.5)
    1602
    

fee spot 0.016501725000000002
fee_future 0.00986544
pnl: -0.030437164999999464
pnl: -0.03828716500000004
pnl: -0.027877164999999735
pnl: -0.027887164999999745
pnl: -0.027877164999999735
pnl: -0.025257165
pnl: -0.02524716500000149
pnl: -0.0330771650000006
pnl: -0.02786716499999967
pnl: -0.027877164999999735
pnl: -0.03048716499999946
pnl: -0.027907165000001153
pnl: -0.027877164999999735
pnl: -0.03309716500000062
pnl: -0.030477164999999393
pnl: -0.025257165
pnl: -0.027857165000001214
pnl: -0.027857165000001214
pnl: -0.030477164999999393
pnl: -0.02786716499999967
pnl: -0.027897165000001143
pnl: -0.027897165000001143
pnl: -0.025287164999999976
pnl: -0.03048716499999946
pnl: -0.027887164999999745
pnl: -0.027887164999999745
pnl: -0.027887164999999745
pnl: -0.0357171650000003
pnl: -0.025297165000001485
pnl: -0.025287164999999976
pnl: -0.027907165000001153
pnl: -0.03050716500000081
pnl: -0.030517165000000877
pnl: -0.02792716499999973
pnl: -0.025317165000000006
pnl: -0.030527164999999388
pnl: -0

In [ ]:
# (curret_spot - price_spot)/price_spot +   -(curret_ft - price_ft)/price_ft
# (float(close_spot['fills'][0]['price']) - price_spot)/price_spot +   -(float(um_futures_client.get_account_trades(symbol=pair, orderId=close_future['orderId'])[0]['price']) - price_ft)/price_ft
# close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
# close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)


In [55]:
# calculate profit
BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])

# outcome_spot = sum([float(i['qty']) * float(i['price']) for i in close_spot['fills']]) #taker
outcome_spot = float(status_spot_close['cummulativeQuoteQty']) #maker

res_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future_id)[0]
res_ft_open = um_futures_client.get_account_trades(symbol=pair, orderId=ft_order_id)[0]

assert float(res_ft_close["qty"]) == quantity_ft and float(res_ft_open["qty"]) == quantity_ft
# commision_close_spot = sum([float(i['commission']) for i in close_spot['fills']])*BNB_price
commision_close_spot = usdt_value_spot * 0.00075
commision_close_future = float(res_ft_close['commission'])*BNB_price
commision_open_future = float(res_ft_open['commission'])*BNB_price

pnl_ft = float(res_ft_close['realizedPnl'])
pnl_spot = outcome_spot -  usdt_value_spot
all_profit = pnl_ft + pnl_spot - commision_close_future - commision_close_spot*2 - commision_open_future
all_profit


-0.0015563970000005154

In [ ]:
# response = um_futures_client.get_account_trades(symbol=pair,orderId=ft_order_id, recvWindow=6000)
# BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])
# float(response[0]['commission']) * BNB_price / usdt_value_ft

In [ ]:
commision_close_future / (0.03931*280)*100